In [1]:
import sys
sys.path.append("../src")

In [2]:
import os
import gc

import cv2
import pyvips
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from utils import *

In [3]:
data_dir = os.path.join("..", "data", "images")
image_names = os.listdir(data_dir)

print("num images:", len(image_names))

num images: 127


In [4]:
image_paths = [os.path.join(data_dir, i) for i in image_names]
images = [pyvips.Image.new_from_file(f, access="sequential") for f in image_paths]

: 

In [5]:
downsample_factor = 100
spacing = 10

for i, image in tqdm(enumerate(images[5:]), desc="Processing images", total=len(images)):
    image = image.numpy()
    downsampled = downsample(image, downsample_factor)
    background_color = get_background_color(downsampled)

    contours = get_contours(downsampled)
    contours = [c * downsample_factor for c in contours]
    coordinates = get_coordinates(contours)
    coordinate_dict = squeeze_coordinates(coordinates, spacing)

    processed_image = organize_tissues(image, spacing, coordinate_dict)

    del image, contours, coordinates, coordinate_dict
    gc.collect()

    is_white = np.all(processed_image == [255, 255, 255], axis=-1)
    is_black = np.all(processed_image == [0, 0, 0], axis=-1)

    background_mask = np.logical_or(is_white, is_black)
    processed_image[background_mask] = background_color

    del is_white, is_black, background_mask
    gc.collect()

    processed_image = downsample(processed_image, downsample_factor)

    fig, axes = plt.subplots(1, 2, figsize=(8, 8))

    axes[0].imshow(downsampled)
    axes[0].set_title(f"original {image_names[i]}")

    axes[1].imshow(processed_image)
    axes[1].set_title(f"subsampled {image_names[i]}")

    plt.show()

Processing images:   0%|          | 0/127 [00:00<?, ?it/s]